In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem

from pymoo.factory import get_crossover, get_mutation, get_sampling
from pymoo.algorithms.soo.nonconvex.ga import GA

from pooling_functions import calc_pool_conc, calc_pools_conc, MinConcsProblem, MinNumCntrProblem

In [3]:
criff_isos = ['BLZ', 'PAN', 'NIC', 'GTM', 'AIA', 'ATG', 'ABW', 'BHS', 'BRB', 'BMU', 
              'VGB', 'CYM', 'DMA', 'HTI', 'JAM', 'MSR', 'KNA', 'LCA', 'VCT', 'SXM', 'TTO', 'TCA']

pcrafi_isos = ['FJI', 'FSM', 'COK', 'KIR', 'MHL', 'NIU', 'PLW', 'PNG', 'WSM', 
               'SLB', 'TLS', 'TON', 'TUV', 'VUT']

ex_pools = [pcrafi_isos] + [criff_isos]

Load data:

In [5]:
datay = pd.read_csv('data/yearly_damages.csv', index_col=0)
datay.head()

,ASM,KHM,COK,FJI,PYF,IDN,KIR,LAO,MYS,MHL,...,MWI,MUS,MOZ,SOM,TZA,ZMB,ZWE,BWA,ZAF,SWZ
0,0.000000e+00,1.174686e+05,61567.633482,0.000000,2066.621255,4.932370e+09,0.0,1.468178e+05,1.363927e+10,0.000000,...,0.0,4.132596e+08,1.240871e+07,1.518563e+07,8.905124e+09,0.0,0.0,0.0,1.297125e+06,0.0
1,0.000000e+00,2.135547e+07,0.000000,0.000000,1817.411519,0.000000e+00,0.0,2.456694e+06,5.648761e+06,81.644624,...,0.0,3.394774e+05,1.486315e+08,0.000000e+00,7.669974e+06,0.0,0.0,0.0,0.000000e+00,0.0
2,2.615973e+07,6.706448e+08,2374.730234,0.000000,42.486543,6.417800e+07,0.0,0.000000e+00,3.386322e+05,0.000000,...,0.0,6.093447e+07,1.342145e+07,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
3,6.236010e+07,3.473528e+07,702363.195433,4825.253640,0.000000,0.000000e+00,0.0,0.000000e+00,2.667397e+09,0.000000,...,0.0,1.005025e+05,1.356082e+04,8.197128e+05,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
4,0.000000e+00,2.488902e+07,0.000000,158.037936,0.000000,3.155947e+09,0.0,0.000000e+00,0.000000e+00,0.000000,...,0.0,5.339540e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0


Define alpha:

In [6]:
RT = 200

alpha = 1-1/RT
alpha

0.995

Load damages by region:

In [7]:
reg_names = ['EAP', 'CAC', 'SA', 'SEA']

regs_datay = {}
bools_regs_datay = {}
i = 0

for reg_name in reg_names[:2]:
    regs = np.loadtxt('data/preprocessing/{}.txt'.format(reg_name), dtype=str)
    
    reg_datay = datay[regs.tolist()]
    new_cols_ord = ex_pools[i] + [col for col in reg_datay.columns.values if not col in ex_pools[i]]

    regs_datay.update({reg_name: reg_datay[new_cols_ord]})
    bools_regs_datay.update({reg_name: reg_datay[new_cols_ord].values >= np.nanquantile(reg_datay[new_cols_ord].values, 
                            alpha, axis=0)})

    i += 1

Current diversification of PCRAFI:

In [8]:
bools = datay[pcrafi_isos].values >= np.nanquantile(datay[pcrafi_isos].values, alpha, axis=0)
1-calc_pools_conc(np.repeat(1, len(pcrafi_isos)), datay[pcrafi_isos].values, bools, alpha, 1)

array([[0.49]])

Current diversification of CCRIF:

In [9]:
bools = datay[criff_isos].values >= np.nanquantile(datay[criff_isos].values, alpha, axis=0)
1-calc_pools_conc(np.repeat(1, len(criff_isos)), datay[criff_isos].values, bools, alpha, 1)

array([[0.48]])

In [10]:
x_pool_int_criff = np.ones(len(criff_isos))
x_pool_int_pcrafi = np.ones(len(pcrafi_isos))

x_fixed_pools = [x_pool_int_pcrafi, x_pool_int_criff]

In [34]:
i = 0
ex_pools_names = ['PCRAFI', 'CRIFF']

for reg in reg_names[:2]:

    print("FIND OPTIMAL REGIONAL EXTENSION FOR {}".format(ex_pools_names[i]))
    print("Step 1 - Extended pool with the minimum concentration")

    num_cntries_to_explore = regs_datay[reg].shape[1]-x_fixed_pools[i].shape[0]

    problem = MinConcsProblem(num_cntries_to_explore, regs_datay[reg], 
                                bools_regs_datay[reg], alpha, 
                                calc_pools_conc, 1, x_fixed_pools[i])

    algorithm = GA(pop_size=reg_datay.shape[1],
                   sampling=get_sampling("bin_random"),
                   crossover=get_crossover("bin_hux"),
                   mutation=get_mutation("bin_bitflip"),
                   eliminate_duplicates=True)

    res_reg = minimize(problem,
               algorithm,
               ('n_gen', 750),
               verbose=False,
               save_history=True,
               seed=1)

    print("Step 2 - Smallest set of countries within this pool maintaining min conc\n")

    X_ex_pool = res_reg.X.astype(int)
    X = np.hstack([x_fixed_pools[i], X_ex_pool])

    reg_datay_opt = regs_datay[reg][regs_datay[reg].columns[X > 0]]
    bools_opt = reg_datay_opt >= np.quantile(reg_datay_opt, alpha, axis=0)

    problem = MinNumCntrProblem(reg_datay_opt, bools_opt.values, alpha, calc_pool_conc, res_reg.F[0])

    algorithm = GA(pop_size=reg_datay_opt.shape[1],
    sampling=get_sampling("bin_random"),
    crossover=get_crossover("bin_hux"),
    mutation=get_mutation("bin_bitflip"),
    eliminate_duplicates=True)

    res_reg2 = minimize(problem,
                         algorithm,
                         ('n_gen', 750),
                         verbose=False,
                         save_history=True,
                         pf=np.array(0.0))

    i+=1

FIND OPTIMAL REGIONAL EXTENSION FOR PCRAFI
Step 1 - Extended pool with the minimum concentration
Step 2 - Smallest set of countries within this pool maintaining min conc

FIND OPTIMAL REGIONAL EXTENSION FOR CRIFF
Step 1 - Extended pool with the minimum concentration
Step 2 - Smallest set of countries within this pool maintaining min conc

